In [ ]:
## Author : Seong-Han

import copy
import json
import random
import io
import distutils.dir_util
import warnings
warnings.filterwarnings("ignore")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
        
from collections import Counter
from tqdm import tqdm

In [ ]:
import sys
sys.path.append('../')
from basic_utils import *

In [ ]:
data_path = '../../../1_melon_playlist/data/'
train = pd.read_json(data_path+'train.json')
val = pd.read_json(data_path+'val.json')
# test = pd.read_json(data_path+'test.json')
song_meta = load_json(data_path+'song_meta.json')
# genre_gn_all = pd.read_json(data_path+'genre_gn_all.json', typ = 'series')
# val_x = pd.read_json('/kaggle/input/kakao-train-valid-split/val_x.json')
# val_y = pd.read_json('/kaggle/input/kakao-train-valid-split/val_y.json')

In [ ]:
no_song, no_both, no_tag, yes_index = check_target_type(val)

In [ ]:
train['date'] = train['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )
train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]))

date_list = list(train['date'].unique())
date_list= sorted(date_list)

date_dict = {}
for i,k in enumerate(date_list):
    date_dict[k] = i

In [ ]:
#### popular monthly tag

train['date'] = train['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )
train['year'] = train['updt_date'].apply(lambda x: int(x[2:4]))

date_list = list(train['date'].unique())
date_list= sorted(date_list)

date_dict = {}
for i,k in enumerate(date_list):
    date_dict[k] = i

popular_date_tag = []
for i in range(len(date_list)):
    if i <= 13:  ## 13이전에는 한 달에 100개가 안나옴
        year = int(str(date_list[i])[0]) ## year
        tmp = train[train['year'] == year]
        _, year_tag = most_popular(tmp, "tags", 100)
        popular_date_tag.append({
            "date" : date_list[i],
            "tags" : year_tag,})
        
    else:
        tmp = train[train['date'] == date_list[i]]
        _, date_tag = most_popular(tmp, "tags", 100)
        popular_date_tag.append({
            "date" : date_list[i],
            "tags" : date_tag,})

In [ ]:
#### popular monthly song

popular_date_song = []
for i in range(len(date_list)):
    if i <= 9:  ## 11이전에는 한 달에 100개가 안나옴
        year = int(str(date_list[i])[0]) ## year
        tmp = train[train['year'] == year]
        _, date_song = most_popular(tmp, "songs", 200)
        popular_date_song.append({
            "date" : date_list[i],
            "songs" : date_song,})               
        
    else:
        tmp = train[train['date'] == date_list[i]]
        _, date_song = most_popular(tmp, "songs", 200)
        popular_date_song.append({
            "date" : date_list[i],
            "songs" : date_song,})         

In [ ]:
## most genral genre
song_meta = load_json('/kaggle/input/kakao-arena-melon/song_meta.json')
song_meta =  {int(song["id"]): song for song in song_meta}

In [ ]:
tag_song_dict = load_json('/kaggle/input/kakao-save-song-tag-dict/tag_song_dict_no0.json')
## https://www.kaggle.com/incastle/kakao-save-song-tag-dict?scriptVersionId=33768264

song_tag_dict = load_json('/kaggle/input/kakao-save-song-tag-dict/song_tag_dict.json')
## https://www.kaggle.com/incastle/kakao-save-song-tag-dict?scriptVersionId=33768264

val['date'] = val['updt_date'].apply(lambda x: int(str(x[2:4]) + str(x[5:7])) )
val['year'] = val['updt_date'].apply(lambda x: int(x[2:4]))

### val2 : 노래가 없는 케이스

In [ ]:
val2 = val[val.index.isin(no_song)]
val2.reset_index(inplace = True, drop = True)
val2.head()

In [ ]:
val2_predict = []
for i in range(len(val2)):
    date = date_dict[val2.loc[i:i, 'date'].values[0] ]

    ############### song 채우기 ##################################
    song_counter = Counter()


    for tag in list(val2.loc[i:i, 'tags'].values[0]):
        if str(tag) in tag_song_dict:
            for song in tag_song_dict[str(tag)]:
                song_counter.update({song : 1})
    song_counter = sorted(song_counter.items(), key= lambda x:x[1], reverse = True)        
    cur_song = []
    for k in song_counter[:100]:
        cur_song.append(k[0])
        
    if (len(cur_song) < 100) & (len(cur_song) > 0 ):
        ## 부족하면 채워준다.
        update_song = remove_seen(cur_song, popular_date_song[date]['songs'])
        cur_song.extend(update_song)
        cur_song = cur_song[:100]
        
    if len(cur_song) == 0:
        cur_song = popular_date_song[date]['songs'][:100]

    #############################################################
    
    
    ############### tag 채우기 ##################################
    tag_counter = Counter()
    for song in list(val2.loc[i:i, 'songs'].values[0]):
        if str(song) in song_tag_dict:
            for tag in song_tag_dict[str(song)]:
                tag_counter.update({tag : 1})
    tag_counter = sorted(tag_counter.items(), key= lambda x:x[1], reverse = True)        
    tag_counter = tag_counter[:100] ## 넉넉히 100개 잡는다. 
    
    cur_tag = []
    for k in tag_counter:
        cur_tag.append(k[0])     
        
    if (len(cur_tag) < 100) & (len(cur_tag) > 0 ):
        ## 부족하면 채워준다.
        update_tag = remove_seen(cur_tag, popular_date_tag[date]['tags'])
        cur_tag.extend(update_tag)
        cur_tag = cur_tag[:100]
    if len(cur_tag) == 0:
        cur_tag = popular_date_tag[date]['tags'][:100]
        
        
    #############################################################       
    

    val2_predict.append({
        "id" : list(val2.loc[i:i, 'id'])[0],
        "songs": cur_song,
        "tags": remove_seen(list(val2.loc[i:i, 'tags'])[0], cur_tag)[:10],
    })

In [ ]:
## save file
write_json(val2_predict, 'val2_predict.json')

### val4 : 태그, 노래 모두 없는 케이스
- 카카오 아레나의 방식으로 추천
- 각 방식마다 나눠서 학습해야하지만 부분적인 성능 확인을 위해서 일단 하나씩 확인

In [ ]:
Validate = val[~val.index.isin(no_tag)]
Validate = Validate[~Validate.index.isin(no_song)]
Validate = Validate[~Validate.index.isin(yes_index)]
Validate.reset_index(inplace = True, drop = True)

In [ ]:
val_predict = []

for i in range(len(Validate)):
    date = date_dict[Validate.loc[i:i, 'date'].values[0]]
    val_predict.append({
        "id" : list(Validate.loc[i:i, 'id'])[0],
        "songs": popular_date_song[date]['songs'][:100],
        "tags": popular_date_tag[date]['tags'][:10],
    })

In [ ]:
write_json(val_predict, 'val4_predict.json')